In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta
import re

code = "009830"
base_url = f"https://finance.naver.com/item/board.naver?code={code}&page=1"

driver = webdriver.Chrome()
driver.get(base_url)
time.sleep(1)

columns = ['일자', '시각', '제목', '본문', '댓글 수', '닉네임', '조회수', '공감수', '비공감수']
result = []

# 현재 날짜에서 1달 전 날짜 계산
one_month_ago = datetime.now() - timedelta(days=30)

stop_collecting = False
current_page = 1

while not stop_collecting:
    # 현재 페이지의 게시글들 수집
    trs = driver.find_elements(By.XPATH, '//*[@id="content"]/div[3]/table[1]/tbody/tr')
    
    for tr in trs[1:]:  # 첫 번째 tr은 헤더이므로 제외
        try:
            row = tr.text
            if "클린봇" in row or "설정" in row or not row.strip():
                continue
            if not row.strip():
                continue

            split_row = row.split()
            date_str = split_row[0]

            try:
                post_date = datetime.strptime(date_str, "%Y.%m.%d")
            except ValueError:
                continue
            
            if post_date < one_month_ago:
                stop_collecting = True
                break

            time_str = split_row[1]
            title_parts = []
            reply_count = 0
            nickname = ''
            views = likes = dislikes = 0

            for item in split_row[2:]:
                if re.match(r'^\[\d+\]$', item):
                    reply_count = int(item.strip('[]'))
                elif '****' in item:
                    nickname = item
                    break
                else:
                    title_parts.append(item)

            if nickname:
                idx_nick = split_row.index(nickname)
                try:
                    views = int(split_row[idx_nick + 1].replace(',', ''))
                    likes = int(split_row[idx_nick + 2])
                    dislikes = int(split_row[idx_nick + 3])
                except (IndexError, ValueError):
                    pass

            title_text = ' '.join(title_parts)

            # 게시글 링크 수집
            post_link = tr.find_element(By.XPATH, './/td[2]/a').get_attribute('href')
            
            # 새 탭에서 본문 페이지 열기
            driver.execute_script(f"window.open('{post_link}');")
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(1)
            
            # 게시글 본문 수집집
            try:
                body_text = driver.find_element(By.XPATH, '//*[@id="body"]').text
            except:
                body_text = "게시글 본문 수집 실패"
            
            result.append([date_str, time_str, title_text, body_text, reply_count, nickname, views, likes, dislikes])

            # 본문 탭 닫고 원래 탭으로 돌아가기
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
        except Exception as e:
            print(f"게시글 수집 중 오류 발생: {e}")
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            continue

    if stop_collecting:
        print("한 달 전 게시글 발견, 수집 종료합니다.")
        break

    # 다음 페이지로 이동
    current_page += 1
    
    if current_page % 10 == 1 and current_page != 1:  # 11, 21, 31... 페이지로 이동
        try:
            next_button = driver.find_element(By.XPATH, '//td[@class="pgR"]/a')
            next_button.click()
            time.sleep(2)
            print(f"{current_page}페이지로 이동했습니다.")
        except:
            print("다음 버튼이 없어 수집 종료합니다.")
            break
    else:  # 2~10, 12~20... 페이지로 이동
        try:
            page_buttons = driver.find_elements(By.XPATH, '//*[@id="content"]/div[3]/table[2]/tbody/tr/td[2]/table/tbody/tr/td')
            for btn in page_buttons:
                if btn.text.strip() == str(current_page):
                    btn.find_element(By.TAG_NAME, 'a').click()
                    time.sleep(1)
                    # print(f"{current_page}페이지로 이동했습니다.")
                    break
        except Exception as e:
            print(f"페이지 이동 실패: {e}")
            break

# 결과 저장
df = pd.DataFrame(result, columns=columns)
display(df)

current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
df.to_csv(f"../data/stock_sentiment_{current_time}.csv", index=False)

driver.quit()


게시글 수집 중 오류 발생: 20개 채움


KeyboardInterrupt: 